# 🚀 Training YOLOv8 on Raw Medical Data (DICOM + CSV)

**The Problem:** You have raw DICOM files and a CSV label file.
**The Solution:** YOLOv8 needs **PNG Images** and **.txt Label Files**.

This notebook will:
1.  **Convert** your DICOMs to PNGs.
2.  **Convert** your CSV labels to YOLO format.
3.  **Train** the model.

In [ ]:
# 1. Install Dependencies
!pip install ultralytics pydicom pandas opencv-python
import ultralytics
from ultralytics import YOLO
import os
import pandas as pd
import pydicom
import cv2
import numpy as np
import shutil

ultralytics.checks()

In [ ]:
# 2. Inspect Your CSV
# We need to know the column names (e.g., x, y, width, height, filename)
csv_path = '/kaggle/input/medical-deepfakes-lung-cancer/labels_exp1.csv'
if os.path.exists(csv_path):
    df = pd.read_csv(csv_path)
    print("First 5 rows of your labels:")
    display(df.head())
else:
    print(f"❌ Could not find CSV at {csv_path}")

In [ ]:
# 3. PREPROCESSING CONFIGURATION (YOU MUST EDIT THIS)
# Look at the output above and update these column names to match your CSV!

COLUMN_FILENAME = 'filename'  # Column containing the DICOM filename
COLUMN_CLASS = 'class'        # Column containing 'Real' or 'Fake'
COLUMN_XMIN = 'xmin'          # Bounding box coordinates
COLUMN_YMIN = 'ymin'
COLUMN_XMAX = 'xmax'
COLUMN_YMAX = 'ymax'

# Output Directories (We create these in the writable /kaggle/working folder)
OUTPUT_DIR = '/kaggle/working/dataset'
IMAGES_DIR = os.path.join(OUTPUT_DIR, 'images/train')
LABELS_DIR = os.path.join(OUTPUT_DIR, 'labels/train')
os.makedirs(IMAGES_DIR, exist_ok=True)
os.makedirs(LABELS_DIR, exist_ok=True)

In [ ]:
# 4. Run Conversion (DICOM -> PNG + CSV -> YOLO)

def convert_to_yolo_bbox(bbox, w, h):
    # Convert (xmin, ymin, xmax, ymax) to (x_center, y_center, width, height) normalized
    x_center = ((bbox[0] + bbox[2]) / 2) / w
    y_center = ((bbox[1] + bbox[3]) / 2) / h
    width = (bbox[2] - bbox[0]) / w
    height = (bbox[3] - bbox[1]) / h
    return x_center, y_center, width, height

print("Starting conversion... this may take a few minutes.")

# Assuming all DICOMs are in one folder
dicom_dir = '/kaggle/input/medical-deepfakes-lung-cancer/CT_Scans'

if os.path.exists(csv_path) and os.path.exists(dicom_dir):
    for index, row in df.iterrows():
        try:
            # 1. Load DICOM
            fname = row[COLUMN_FILENAME]
            if not fname.endswith('.dcm'): fname += '.dcm'
            dcm_path = os.path.join(dicom_dir, fname)
            
            if not os.path.exists(dcm_path): continue
            
            ds = pydicom.dcmread(dcm_path)
            img = ds.pixel_array
            
            # Normalize to 0-255
            img = (img - img.min()) / (img.max() - img.min() + 1e-5) * 255
            img = img.astype(np.uint8)
            h, w = img.shape
            
            # 2. Save as PNG
            png_filename = fname.replace('.dcm', '.png')
            cv2.imwrite(os.path.join(IMAGES_DIR, png_filename), img)
            
            # 3. Create YOLO Label
            # Class: 0 for Fake, 1 for Real (Adjust based on your CSV)
            cls = 0 if row[COLUMN_CLASS] == 'Fake' else 1
            
            bbox = [row[COLUMN_XMIN], row[COLUMN_YMIN], row[COLUMN_XMAX], row[COLUMN_YMAX]]
            xc, yc, bw, bh = convert_to_yolo_bbox(bbox, w, h)
            
            label_path = os.path.join(LABELS_DIR, png_filename.replace('.png', '.txt'))
            with open(label_path, 'w') as f:
                f.write(f"{cls} {xc} {yc} {bw} {bh}\n")
                
        except Exception as e:
            print(f"Error processing {row[COLUMN_FILENAME]}: {e}")
            
    print("Conversion Complete! ✅")
else:
    print("Could not find DICOM dir or CSV.")

In [ ]:
# 5. Create data.yaml
yaml_content = f"""
train: {IMAGES_DIR}
val: {IMAGES_DIR}  # Using same for val just to make it run (ideally split them)

nc: 2
names: ['Fake', 'Real']
"""

with open('/kaggle/working/data.yaml', 'w') as f:
    f.write(yaml_content)
    
print("Created data.yaml")

In [ ]:
# 6. Train YOLOv8
model = YOLO('yolov8s.pt')

results = model.train(
    data='/kaggle/working/data.yaml', 
    epochs=50, 
    imgsz=512, 
    batch=16,
    name='medical_deepfake_detector'
)

In [ ]:
# 7. Export Weights
import shutil
import glob

run_dirs = sorted(glob.glob('runs/detect/medical_deepfake_detector*'), key=os.path.getmtime)
if run_dirs:
    latest_run = run_dirs[-1]
    source_path = os.path.join(latest_run, 'weights', 'best.pt')
    destination_path = '/kaggle/working/best.pt'
    if os.path.exists(source_path):
        shutil.copy(source_path, destination_path)
        print(f"✅ Success! Download 'best.pt' from the Output tab.")